In [16]:
import pandas as pd
from pathlib import Path
import os
import sys

import matplotlib.pyplot as plt

sys.path.append('../src')


In [32]:
file_path = Path('data') / 'beautyamazon.csv'
df = pd.read_csv(file_path)

In [33]:
df.head()

,UserId,ProductId,ProductType,Rating,Timestamp,URL
0,A3NHUQ33CFH3VM,B00LLPT4HI,Eyeliner & Kajal,5,1405814400,https://www.amazon.in/Maybelline-Colossal-Kaja...
1,A1TIRNQ7O4REOH,B00LLPT4HI,Eyeliner & Kajal,4,1405987200,https://www.amazon.in/Maybelline-Colossal-Kaja...
2,A2Y36BR4YSY9F7,B00LLPT4HI,Eyeliner & Kajal,5,1405728000,https://www.amazon.in/Maybelline-Colossal-Kaja...
3,A23H6FAOLEMAKC,B00LLPT4HI,Eyeliner & Kajal,5,1405814400,https://www.amazon.in/Maybelline-Colossal-Kaja...
4,A3CHYZGF3OO6WD,B00LLPT4HI,Eyeliner & Kajal,5,1405641600,https://www.amazon.in/Maybelline-Colossal-Kaja...


In [34]:
print(df.shape)

(1348246, 6)


In [36]:
df = df[['UserId', 'ProductId', 'Timestamp']]

In [37]:
df = df.rename(columns={'UserId':'user_id','ProductId': 'item_id'})

In [38]:
df.user_id.nunique(), df.item_id.nunique()

(883753, 23838)

In [39]:
df.groupby('user_id').size().describe()

count    883753.000000
mean          1.525591
std           2.008556
min           1.000000
25%           1.000000
50%           1.000000
75%           1.000000
max         222.000000
dtype: float64

In [40]:
df.groupby('item_id').size().describe()

count    23838.000000
mean        56.558688
std         92.422208
min         16.000000
25%         21.000000
50%         31.000000
75%         56.000000
max       2869.000000
dtype: float64

In [41]:
def filter_users(df, user_min_count, user_col='user_id'):
    print('Filtering users based on total interactions...')

    # Count total interactions per user (not unique items)
    user_count = df.groupby(user_col).size()

    # Keep users with at least `user_min_count` interactions
    user_ids = user_count[user_count >= user_min_count].index

    print(f'Number of users before: {len(user_count)}')
    print(f'Number of users after: {len(user_ids)}')

    print(f'Interactions before: {len(df)}')
    df = df[df[user_col].isin(user_ids)]
    print(f'Interactions after: {len(df)}')

    return df




In [44]:
df = filter_users(df, user_min_count=5)

Filtering users based on total interactions...
Number of users before: 45390
Number of users after: 28040
Interactions before: 313001
Interactions after: 243601


In [45]:
df.groupby('user_id').size().describe()

count    28040.000000
mean         8.687625
std          7.831627
min          5.000000
25%          5.000000
50%          6.000000
75%          9.000000
max        222.000000
dtype: float64

In [46]:
df.groupby('item_id').size().describe()

count    22445.000000
mean        10.853241
std         19.707212
min          1.000000
25%          3.000000
50%          5.000000
75%         10.000000
max        442.000000
dtype: float64

In [47]:
df = df.sort_values(['user_id', 'Timestamp'])

In [48]:
df.user_id = df.user_id.astype('category').cat.codes + 1
df.item_id = df.item_id.astype('category').cat.codes + 1

In [51]:
import os

# Create the directory if it doesn't exist
os.makedirs('data', exist_ok=True)

# Now save the file
df[['user_id', 'item_id']].to_csv('data/beauty2.txt', sep=' ', index=False, header=None)
